In [1]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import SimpleRNN,GRU,LSTM
from keras.models import Sequential
import plotly
from plotly.graph_objs import Scatter, Layout
import plotly.graph_objs as go
# Set a random seed to reproduce the results
np.random.seed(1234)

Using Theano backend.
Using gpu device 0: GeForce GTX 960 (CNMeM is disabled, cuDNN 4007)


In [2]:
# Load the volume data
volume_data = pd.read_csv('../data/volume_data.csv', header=None)
scaler = None
all_hfs = pd.read_csv('../data/hf_list.csv')

def find_index_hf(hf_no):
    return all_hfs[all_hfs['HF'] == hf_no].index.tolist()[0]

In [3]:
maxs = []
mins = []
means = []
stds = []

In [4]:
# Return a training and test data for a site.
def train_test_traffic_data(sitenum, sequence_length, horizon, standardise=True):
    global maxs
    global mins
    global means
    global stds
    maxs = []
    mins = []
    means = []
    stds = []
    site_traffic = volume_data[find_index_hf(sitenum)]
    # Keep data till 30th June 2012
    site_traffic= site_traffic[:-2400]
    site_traffic = site_traffic.replace(0, np.nan) 
    site_traffic = site_traffic.interpolate().values 
    
    if horizon == 30:
        site_traffic = site_traffic[0::2] + site_traffic[1::2] 
    elif horizon == 45:
        site_traffic = site_traffic[0::3] + site_traffic[1::3] + site_traffic[2::3]
        
    mean_t = site_traffic.mean()
    max_t = site_traffic.max()
    min_t = site_traffic.min()
    std_t = np.std(site_traffic)
    print("Max-",max_t,"-mean-",mean_t, "-min-",min_t)
    mins.append(min_t)
    maxs.append(max_t)
    means.append(mean_t)
    stds.append(std_t)
    
    if standardise: 
        site_traffic = (site_traffic - mean_t) / std_t
    result = []
    for index in range(len(site_traffic) - sequence_length):
        result.append(site_traffic[index: index + sequence_length])
    result = np.array(result)
    # We test only last 30 days = 30 * 96 = 2880
    row = result.shape[0] - 2880
    train = result[:row, :]
    np.random.shuffle(train)
    X_train = train[:, :-1]
    y_train = train[:, -1]
    X_test = result[row:, :-1]
    y_test = result[row:, -1]

    return [X_train, y_train, X_test, y_test]

In [5]:
def simple_rnn_model():
    mdl = Sequential()
    # a network with 1-dimensional input,
    # two hidden layers of sizes 100 and 100
    # and eventually a 1-dimensional output layer
    layers = [1, 100, 100, 1]

    # We also add 10% Dropout in this layer.
    mdl.add(SimpleRNN(
        input_dim=layers[0],
        output_dim=layers[1],
        return_sequences=True))
    mdl.add(Dropout(0.1))

    # 2nd hidden layer
    mdl.add(SimpleRNN(
        layers[2],
        return_sequences=False))
    mdl.add(Dropout(0.1))

    # last layer we use is a Dense layer ( = feedforward).
    # Since we are doing a regression, its activation is linear
    mdl.add(Dense(
        output_dim=layers[3]))
    mdl.add(Activation("linear"))

    start = time.time()
    mdl.compile(loss="mse", optimizer="adam")
    print("Compilation Time : ", time.time() - start)
    return mdl

In [6]:
def gru_model():
    mdl = Sequential()
    # a network with 1-dimensional input,
    # two hidden layers of sizes 50 and 100
    # and eventually a 1-dimensional output layer
    layers = [1, 100, 100, 1]

    # We also add 20% Dropout in this layer.
    mdl.add(GRU(
        input_dim=layers[0],
        output_dim=layers[1],
        return_sequences=True))
    mdl.add(Dropout(0.2))

    # 2nd hidden layer
    mdl.add(GRU(
        layers[2],
        return_sequences=False))
    mdl.add(Dropout(0.2))
    

    # last layer we use is a Dense layer ( = feedforward).
    # Since we are doing a regression, its activation is linear
    mdl.add(Dense(
        output_dim=layers[3]))
    mdl.add(Activation("linear"))

    start = time.time()
    mdl.compile(loss="mse", optimizer="adam")
    print("Compilation Time : ", time.time() - start)
    return mdl

In [7]:
def lstm_model():
    mdl = Sequential()
    # a network with 1-dimensional input,
    # two hidden layers of sizes 50 and 100
    # and eventually a 1-dimensional output layer
    layers = [1, 100, 200, 1]

    # We also add 20% Dropout in this layer.
    mdl.add(LSTM(
        input_dim=layers[0],
        output_dim=layers[1],
        return_sequences=True))
    mdl.add(Dropout(0.2))

    # 2nd hidden layer
    mdl.add(LSTM(
        layers[2],
        return_sequences=False))
    mdl.add(Dropout(0.2))
    
    # last layer we use is a Dense layer ( = feedforward).
    # Since we are doing a regression, its activation is linear
    mdl.add(Dense(
        output_dim=layers[3]))
    mdl.add(Activation("linear"))

    start = time.time()
    mdl.compile(loss="mse", optimizer="adam")
    print("Compilation Time : ", time.time() - start)
    return mdl

In [8]:
def run_network(data, mdl_type):
    global_start_time = time.time()
    epochs = 20
    
    X_train, y_train, X_test, y_test = data
    
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    print('\nData Loaded. Compiling...\n')

    if mdl_type == 'rnn':
        mdl = simple_rnn_model()
    elif mdl_type == 'gru':
        mdl = gru_model()
    elif mdl_type == 'lstm':
        mdl = lstm_model()
    else:
        return
       
    try:
        mdl.fit(X_train, y_train, batch_size=512,
                nb_epoch=epochs, validation_split=0.05)
        predicted_trffic = mdl.predict(X_test)
        predicted_trffic = np.reshape(predicted_trffic, (predicted_trffic.size,))
    except KeyboardInterrupt:
        print('Training duration (s) : ', time.time() - global_start_time)
        return mdl, y_test, 0

    print('Training duration (s) : ', time.time() - global_start_time)
    
    y_test = y_test * stds[0] + means[0]
    predicted_trffic = predicted_trffic * stds[0] + means[0]
    
    return mdl, y_test, predicted_trffic

In [9]:
from sklearn.svm import SVR
def svm_model(data):
    global_start_time = time.time()
    
    X_train, y_train, X_test, y_test = data

    print('\nData Loaded..., training model\n')

    mdl = SVR(cache_size=2048)
        
    try:
        mdl.fit(X_train, y_train)
        predicted_trffic = mdl.predict(X_test)
    except KeyboardInterrupt:
        print('Training duration (s) : ', time.time() - global_start_time)
        return mdl, y_test, 0

    print('Training duration (s) : ', time.time() - global_start_time)

    y_test = y_test * stds[0] + means[0]
    predicted_trffic = predicted_trffic * stds[0] + means[0]
    
    return mdl, y_test, predicted_trffic

In [10]:
from sklearn.neighbors import KNeighborsRegressor
def knn_model(data):
    global_start_time = time.time()
  
    X_train, y_train, X_test, y_test = data
    
    print('\nData Loaded...training model\n')

    mdl = KNeighborsRegressor(5, weights='distance')
        
    try:
        mdl.fit(X_train, y_train)
        predicted_trffic = mdl.predict(X_test)
    except KeyboardInterrupt:
        print('Training duration (s) : ', time.time() - global_start_time)
        return mdl, y_test, 0

    print('Training duration (s) : ', time.time() - global_start_time)
        
    return mdl, y_test, predicted_trffic

In [11]:
def plot_predictions(y_test, predicted, horizon):
    t = pd.date_range('6/1/2012', freq=(str(horizon)+'Min'), periods=(30*96*15)/horizon)
    trace0 = go.Scatter(
        x = t,
        y = y_test,
        name = 'Actual')
    trace1 = go.Scatter(
        x = t,
        y = predicted,
        name = 'Predicted')
    data = [trace0, trace1]

    layout = dict(xaxis = dict(title = 'Time'),
                  yaxis = dict(title = 'Volume'),
                  width = 600, height = 450)

    fig = dict(data=data, layout=layout)
    plotly.offline.plot(fig)

In [12]:
import math
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def print_scores(actual, predictions):
    print("MAE=", mean_absolute_error(actual, predictions))
    print("RMSE=", math.sqrt(mean_squared_error(actual, predictions)))
    print("MAPE=", mean_absolute_percentage_error(actual, predictions))

In [13]:
data = train_test_traffic_data(16913, 50, 15)

Max- 908.0 -mean- 224.104820302 -min- 1.0


In [14]:
# Simple RNN
model, y_test, predicted = run_network(data, 'rnn')
print_scores(y_test, predicted)
plot_predictions(y_test, predicted, 15)


Data Loaded. Compiling...

Compilation Time :  0.08252692222595215
Train on 180346 samples, validate on 9492 samples
Epoch 1/20
180346/180346 [==============================] - 14s - loss: 0.1349 - val_loss: 0.0718
Epoch 2/20
180346/180346 [==============================] - 14s - loss: 0.0888 - val_loss: 0.0751
Epoch 3/20
180346/180346 [==============================] - 14s - loss: 0.0820 - val_loss: 0.0703
Epoch 4/20
180346/180346 [==============================] - 14s - loss: 0.0788 - val_loss: 0.0700
Epoch 5/20
180346/180346 [==============================] - 14s - loss: 0.0774 - val_loss: 0.0687
Epoch 6/20
180346/180346 [==============================] - 14s - loss: 0.0763 - val_loss: 0.0676
Epoch 7/20
180346/180346 [==============================] - 14s - loss: 0.0755 - val_loss: 0.0691
Epoch 8/20
180346/180346 [==============================] - 14s - loss: 0.0751 - val_loss: 0.0675
Epoch 9/20
180346/180346 [==============================] - 14s - loss: 0.0748 - val_loss: 0.0680


In [15]:
# GRU
model, y_test, predicted = run_network(data, 'gru')
print_scores(y_test, predicted)
plot_predictions(y_test, predicted, 15)


Data Loaded. Compiling...

Compilation Time :  0.13126182556152344
Train on 180346 samples, validate on 9492 samples
Epoch 1/20
180346/180346 [==============================] - 55s - loss: 0.0963 - val_loss: 0.0712
Epoch 2/20
180346/180346 [==============================] - 55s - loss: 0.0801 - val_loss: 0.0687
Epoch 3/20
180346/180346 [==============================] - 55s - loss: 0.0768 - val_loss: 0.0699
Epoch 4/20
180346/180346 [==============================] - 55s - loss: 0.0754 - val_loss: 0.0685
Epoch 5/20
180346/180346 [==============================] - 55s - loss: 0.0743 - val_loss: 0.0670
Epoch 6/20
180346/180346 [==============================] - 55s - loss: 0.0737 - val_loss: 0.0666
Epoch 7/20
180346/180346 [==============================] - 55s - loss: 0.0731 - val_loss: 0.0673
Epoch 8/20
180346/180346 [==============================] - 55s - loss: 0.0727 - val_loss: 0.0660
Epoch 9/20
180346/180346 [==============================] - 55s - loss: 0.0721 - val_loss: 0.0666


In [16]:
# LSTM
model, y_test, predicted = run_network(data, 'lstm')
print_scores(y_test, predicted)
plot_predictions(y_test, predicted, 15)


Data Loaded. Compiling...

Compilation Time :  0.14759397506713867
Train on 180346 samples, validate on 9492 samples
Epoch 1/20
180346/180346 [==============================] - 95s - loss: 0.0974 - val_loss: 0.0719
Epoch 2/20
180346/180346 [==============================] - 95s - loss: 0.0788 - val_loss: 0.0687
Epoch 3/20
180346/180346 [==============================] - 95s - loss: 0.0762 - val_loss: 0.0673
Epoch 4/20
180346/180346 [==============================] - 95s - loss: 0.0746 - val_loss: 0.0669
Epoch 5/20
180346/180346 [==============================] - 95s - loss: 0.0739 - val_loss: 0.0670
Epoch 6/20
180346/180346 [==============================] - 95s - loss: 0.0732 - val_loss: 0.0666
Epoch 7/20
180346/180346 [==============================] - 95s - loss: 0.0724 - val_loss: 0.0663
Epoch 8/20
180346/180346 [==============================] - 95s - loss: 0.0719 - val_loss: 0.0654
Epoch 9/20
180346/180346 [==============================] - 95s - loss: 0.0709 - val_loss: 0.0656


In [18]:
# KNN
data = train_test_traffic_data(16913, 50, 15, False)
model, y_test, predicted = knn_model(data)
print_scores(y_test, predicted)
plot_predictions(y_test, predicted, 15)

Max- 908.0 -mean- 224.104820302 -min- 1.0

Data Loaded...training model

Training duration (s) :  8.958134889602661
MAE= 19.128982928
RMSE= 26.003481713435214
MAPE= 13.3646591029


In [13]:
data = train_test_traffic_data(16913, 100, 30)

Max- 1710.0 -mean- 448.209640604 -min- 2.0


In [14]:
# Simple RNN
model, y_test, predicted = run_network(data, 'rnn')
print_scores(y_test, predicted)
plot_predictions(y_test, predicted, 30)


Data Loaded. Compiling...

Compilation Time :  0.08630728721618652
Train on 88733 samples, validate on 4671 samples
Epoch 1/20
88733/88733 [==============================] - 13s - loss: 0.1571 - val_loss: 0.0577
Epoch 2/20
88733/88733 [==============================] - 13s - loss: 0.0819 - val_loss: 0.0508
Epoch 3/20
88733/88733 [==============================] - 13s - loss: 0.0696 - val_loss: 0.0519
Epoch 4/20
88733/88733 [==============================] - 13s - loss: 0.0639 - val_loss: 0.0490
Epoch 5/20
88733/88733 [==============================] - 13s - loss: 0.0607 - val_loss: 0.0502
Epoch 6/20
88733/88733 [==============================] - 13s - loss: 0.0586 - val_loss: 0.0485
Epoch 7/20
88733/88733 [==============================] - 13s - loss: 0.0562 - val_loss: 0.0492
Epoch 8/20
88733/88733 [==============================] - 13s - loss: 0.0547 - val_loss: 0.0469
Epoch 9/20
88733/88733 [==============================] - 13s - loss: 0.0536 - val_loss: 0.0447
Epoch 10/20
88733/8

In [15]:
# GRU
model, y_test, predicted = run_network(data, 'gru')
print_scores(y_test, predicted)
plot_predictions(y_test, predicted, 30)


Data Loaded. Compiling...

Compilation Time :  0.13219261169433594
Train on 88733 samples, validate on 4671 samples
Epoch 1/20
88733/88733 [==============================] - 53s - loss: 0.1287 - val_loss: 0.0619
Epoch 2/20
88733/88733 [==============================] - 53s - loss: 0.0719 - val_loss: 0.0565
Epoch 3/20
88733/88733 [==============================] - 53s - loss: 0.0657 - val_loss: 0.0518
Epoch 4/20
88733/88733 [==============================] - 53s - loss: 0.0620 - val_loss: 0.0504
Epoch 5/20
88733/88733 [==============================] - 53s - loss: 0.0602 - val_loss: 0.0530
Epoch 6/20
88733/88733 [==============================] - 53s - loss: 0.0589 - val_loss: 0.0494
Epoch 7/20
88733/88733 [==============================] - 53s - loss: 0.0578 - val_loss: 0.0501
Epoch 8/20
88733/88733 [==============================] - 53s - loss: 0.0566 - val_loss: 0.0489
Epoch 9/20
88733/88733 [==============================] - 53s - loss: 0.0556 - val_loss: 0.0468
Epoch 10/20
88733/8

In [16]:
# LSTM
model, y_test, predicted = run_network(data, 'lstm')
print_scores(y_test, predicted)
plot_predictions(y_test, predicted, 30)


Data Loaded. Compiling...

Compilation Time :  0.17961931228637695
Train on 88733 samples, validate on 4671 samples
Epoch 1/20
88733/88733 [==============================] - 92s - loss: 0.1270 - val_loss: 0.0593
Epoch 2/20
88733/88733 [==============================] - 92s - loss: 0.0680 - val_loss: 0.0514
Epoch 3/20
88733/88733 [==============================] - 92s - loss: 0.0614 - val_loss: 0.0497
Epoch 4/20
88733/88733 [==============================] - 92s - loss: 0.0570 - val_loss: 0.0472
Epoch 5/20
88733/88733 [==============================] - 92s - loss: 0.0545 - val_loss: 0.0463
Epoch 6/20
88733/88733 [==============================] - 93s - loss: 0.0527 - val_loss: 0.0436
Epoch 7/20
88733/88733 [==============================] - 93s - loss: 0.0509 - val_loss: 0.0432
Epoch 8/20
88733/88733 [==============================] - 92s - loss: 0.0502 - val_loss: 0.0441
Epoch 9/20
88733/88733 [==============================] - 92s - loss: 0.0497 - val_loss: 0.0448
Epoch 10/20
88733/8

In [17]:
# KNN
data = train_test_traffic_data(16913, 100, 30, False)
model, y_test, predicted = knn_model(data)
print_scores(y_test, predicted)
plot_predictions(y_test, predicted, 30)

Max- 1710.0 -mean- 448.209640604 -min- 2.0

Data Loaded...training model

Training duration (s) :  5.273013114929199
MAE= 33.4246578007
RMSE= 48.571043792516285
MAPE= 12.3650768686


In [18]:
data = train_test_traffic_data(16913, 50, 45)

Max- 2346.0 -mean- 672.314460906 -min- 3.0


In [19]:
# Simple RNN
model, y_test, predicted = run_network(data, 'rnn')
print_scores(y_test, predicted)
plot_predictions(y_test, predicted, 45)


Data Loaded. Compiling...

Compilation Time :  0.10337376594543457
Train on 58259 samples, validate on 3067 samples
Epoch 1/20
58259/58259 [==============================] - 4s - loss: 0.2727 - val_loss: 0.0629
Epoch 2/20
58259/58259 [==============================] - 4s - loss: 0.1188 - val_loss: 0.0595
Epoch 3/20
58259/58259 [==============================] - 4s - loss: 0.0920 - val_loss: 0.0557
Epoch 4/20
58259/58259 [==============================] - 4s - loss: 0.0789 - val_loss: 0.0545
Epoch 5/20
58259/58259 [==============================] - 4s - loss: 0.0715 - val_loss: 0.0527
Epoch 6/20
58259/58259 [==============================] - 4s - loss: 0.0663 - val_loss: 0.0504
Epoch 7/20
58259/58259 [==============================] - 4s - loss: 0.0647 - val_loss: 0.0500
Epoch 8/20
58259/58259 [==============================] - 4s - loss: 0.0599 - val_loss: 0.0480
Epoch 9/20
58259/58259 [==============================] - 4s - loss: 0.0578 - val_loss: 0.0487
Epoch 10/20
58259/58259 [===

In [20]:
# GRU
model, y_test, predicted = run_network(data, 'gru')
print_scores(y_test, predicted)
plot_predictions(y_test, predicted, 45)


Data Loaded. Compiling...

Compilation Time :  0.13346290588378906
Train on 58259 samples, validate on 3067 samples
Epoch 1/20
58259/58259 [==============================] - 18s - loss: 0.1543 - val_loss: 0.0639
Epoch 2/20
58259/58259 [==============================] - 18s - loss: 0.0731 - val_loss: 0.0571
Epoch 3/20
58259/58259 [==============================] - 17s - loss: 0.0661 - val_loss: 0.0525
Epoch 4/20
58259/58259 [==============================] - 18s - loss: 0.0618 - val_loss: 0.0504
Epoch 5/20
58259/58259 [==============================] - 17s - loss: 0.0584 - val_loss: 0.0486
Epoch 6/20
58259/58259 [==============================] - 17s - loss: 0.0569 - val_loss: 0.0478
Epoch 7/20
58259/58259 [==============================] - 17s - loss: 0.0545 - val_loss: 0.0476
Epoch 8/20
58259/58259 [==============================] - 17s - loss: 0.0535 - val_loss: 0.0458
Epoch 9/20
58259/58259 [==============================] - 17s - loss: 0.0531 - val_loss: 0.0455
Epoch 10/20
58259/5

In [21]:
# LSTM
model, y_test, predicted = run_network(data, 'lstm')
print_scores(y_test, predicted)
plot_predictions(y_test, predicted, 45)


Data Loaded. Compiling...

Compilation Time :  0.14941620826721191
Train on 58259 samples, validate on 3067 samples
Epoch 1/20
58259/58259 [==============================] - 30s - loss: 0.1636 - val_loss: 0.0711
Epoch 2/20
58259/58259 [==============================] - 30s - loss: 0.0732 - val_loss: 0.0560
Epoch 3/20
58259/58259 [==============================] - 30s - loss: 0.0637 - val_loss: 0.0504
Epoch 4/20
58259/58259 [==============================] - 30s - loss: 0.0580 - val_loss: 0.0477
Epoch 5/20
58259/58259 [==============================] - 30s - loss: 0.0552 - val_loss: 0.0460
Epoch 6/20
58259/58259 [==============================] - 30s - loss: 0.0527 - val_loss: 0.0462
Epoch 7/20
58259/58259 [==============================] - 30s - loss: 0.0515 - val_loss: 0.0431
Epoch 8/20
58259/58259 [==============================] - 30s - loss: 0.0493 - val_loss: 0.0422
Epoch 9/20
58259/58259 [==============================] - 30s - loss: 0.0483 - val_loss: 0.0413
Epoch 10/20
58259/5

In [22]:
# KNN
data = train_test_traffic_data(16913, 50, 45, False)
model, y_test, predicted = knn_model(data)
print_scores(y_test, predicted)
plot_predictions(y_test, predicted, 45)

Max- 2346.0 -mean- 672.314460906 -min- 3.0

Data Loaded...training model

Training duration (s) :  1.2333686351776123
MAE= 42.8151428653
RMSE= 62.914296541672456
MAPE= 9.72133679137


In [23]:
# SVR
data = train_test_traffic_data(16913, 50, 30)
model, y_test, predicted = svm_model(data)
print_scores(y_test, predicted)
plot_predictions(y_test, predicted, 30)

Max- 1710.0 -mean- 448.209640604 -min- 2.0

Data Loaded..., training model

Training duration (s) :  409.82636618614197
MAE= 28.6411250263
RMSE= 39.91936899089441
MAPE= 9.00352679977


In [24]:
# SVR
data = train_test_traffic_data(16913, 50, 45)
model, y_test, predicted = svm_model(data)
print_scores(y_test, predicted)
plot_predictions(y_test, predicted, 45)

Max- 2346.0 -mean- 672.314460906 -min- 3.0

Data Loaded..., training model

Training duration (s) :  163.15964102745056
MAE= 40.5185497149
RMSE= 55.69833789919948
MAPE= 8.45540462301


In [25]:
# SVR
# data = train_test_traffic_data(16913, 50, 15)
# model, y_test, predicted = svm_model(data)
# print_scores(y_test, predicted)
# plot_predictions(y_test, predicted, 15)